In [2]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

#实践部分
import struct
import os
#fucntion to load MNIST data
def load_mnist_data(path,kind='train'):
    label_path = os.path.join(path, '%s-labels.idx1-ubyte'%kind)
    image_path = os.path.join(path, '%s-images.idx3-ubyte'%kind)
    with open(label_path,'rb') as lbpath: # open label file
        struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath,dtype=np.uint8)
    with open(image_path,'rb') as imgpath:# open image file
        struct.unpack('>IIII', imgpath.read(16))
        #transform image into 784-dimensional feature vector
        images = np.fromfile(imgpath,dtype=np.uint8).reshape(len(labels),784)
    return images,labels

#定义图片显示函数
import matplotlib.pyplot as plt
def show_image(image):
    plt.figure()
    img = image.reshape(28,28)
    plt.imshow(img, 'gray')
    plt.show()
#数据读取和预处理
from sklearn.preprocessing import StandardScaler
path = 'MNIST'
train_images, train_labels = load_mnist_data(path,kind='train')
# show_image(train_images)
train_y = np.zeros((len(train_labels),10))
for i in range(len(train_labels)):
    train_y[i,train_labels[i]]=1
scaler = StandardScaler()
train_x = scaler.fit_transform(train_images)
test_images, test_labels = load_mnist_data(path,kind='t10k')
# show_image(test_images)
test_y = np.zeros((len(test_labels),10))
for i in range(len(test_labels)):
    test_y[i,test_labels[i]]=1
test_x = scaler.fit_transform(test_images)


# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7, random_state=42)

# 数据预处理：标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 训练和评估模型
def train_and_evaluate(hidden_layer_sizes, activation):
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, max_iter=100, random_state=1)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# 比较不同层数的模型
accuracy_1_layer = train_and_evaluate((50,), 'relu')
accuracy_2_layers = train_and_evaluate((50, 50), 'relu')

print("Accuracy with 1 hidden layer:", accuracy_1_layer)
print("Accuracy with 2 hidden layers:", accuracy_2_layers)

# 比较不同激活函数的模型
accuracy_relu = train_and_evaluate((50,), 'relu')
accuracy_tanh = train_and_evaluate((50,), 'tanh')

print("Accuracy with ReLU activation:", accuracy_relu)
print("Accuracy with Tanh activation:", accuracy_tanh)


NameError: name 'X' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# 加载并准备MNIST数据集
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# 定义一个函数来创建模型
def create_model(layers, activation):
    model = Sequential()
    model.add(Flatten(input_shape=(28 * 28,)))
    for layer_size in layers:
        model.add(Dense(layer_size, activation=activation))
    model.add(Dense(10, activation='softmax'))
    return model

# 比较不同层数和激活函数的模型
layer_configs = [(32,), (64, 32), (128, 64, 32)]  # 不同的层配置
activations = ['relu', 'sigmoid', 'tanh']  # 不同的激活函数
for layers in layer_configs:
    for activation in activations:
        model = create_model(layers, activation)
        model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(train_images, train_labels, epochs=5, batch_size=128, validation_split=0.2)
        test_loss, test_acc = model.evaluate(test_images, test_labels)
        print(f'Layers: {layers}, Activation: {activation}, Test accuracy: {test_acc}')
